# Table of Content
- [Imports](#imports)
- [Resources](#resources)
- [Load Data Sets](#load-data-sets)
- [Explore Data](#explore-data)
	- [Create loyalty flag](#create-loyalty-flag)
	- [Spending habits of the different customer groups
Do the prices differ between customer groups?](#spending-habits-of-the-different-customer-groups
do-the-prices-differ-between-customer-groups?)
	- [Spending flag](#spending-flag)
	- [Order frequency flag](#order-frequency-flag)
- [Export](#export)


## Imports [#](#table-of-content)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import da_helper as da

## Resources [#](#table-of-content)

In [2]:
# project folder
project_folder = Path(r"C:\Users\vynde\Desktop\CareerFoundry Data Analytics\Data Immersion - 4 Python Fundamentals for Data Analysts\Instacart_Basket_Analysis")

# resource folders
prepared_data_folder = project_folder / "02_Data" / "Prepared_Data"

# input files
orders_products_merged_data_file = prepared_data_folder / "orders_products_merged.pkl"
orders_products_merged_updated_data_file = prepared_data_folder / "orders_products_merged_updated.pkl"

# output files
orders_products_merged_updated_v2_data_file = prepared_data_folder / "orders_products_merged_updated_v2.pkl"

## Load Data Sets [#](#table-of-content)

In [3]:
# load data sets
df = pd.read_pickle(orders_products_merged_updated_data_file)

In [4]:
df.head(2)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,busiest_period_of_day
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regulary busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy days,Fewest orders


## Explore Data [#](#table-of-content)

Compare mean order number for whole dataframe and a subset of it

In [5]:
df1 = df[:1000000].groupby("department_id").agg({"order_number": "mean"})
df2 = df.groupby("department_id").agg({"order_number": "mean"})

pd.concat([df2, df1], axis=1)

,order_number,order_number
department_id,,
1,15.457838,NaN
2,17.277920,NaN
3,17.170395,NaN
4,17.811403,18.825780
5,15.215751,NaN
6,16.439806,NaN
7,17.225802,17.472355
8,15.340650,NaN
9,15.895474,NaN


>Ofcourse the mean values differ from each other, because different Series were used for the calculation.<br>
We can also see that there are only 8 of 21 different department_ids in the first million records.

##### Create loyalty flag [#](#table-of-content)

In [6]:
# create column with max order count for each user
df["max_order"] = df.groupby("user_id")["order_number"].transform("max")

In [7]:
# create loyalty_lag column
da.create_flag(df, "max_order", "loyalty_flag", [10, 40], ["New customer", "Regular customer", "Loyal customer"])
df[df.columns[-6:]].head() # show last 6 columns

,prices,price_label,busiest_day,busiest_period_of_day,max_order,loyalty_flag
0,9.0,Mid-range product,Regulary busy,Average orders,10,New customer
1,9.0,Mid-range product,Least busy days,Fewest orders,10,New customer
2,9.0,Mid-range product,Least busy days,Average orders,10,New customer
3,9.0,Mid-range product,Least busy days,Fewest orders,10,New customer
4,9.0,Mid-range product,Least busy days,Average orders,10,New customer


In [8]:
df["loyalty_flag"].value_counts()

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

##### Spending habits of the different customer groups
Do the prices differ between customer groups? [#](#table-of-content)

In [9]:
# prices summary for different customer groups
df.groupby("loyalty_flag").agg({"prices": ["mean", "std", "min", "max"]})

prices                          
                       mean         std  min      max
loyalty_flag                                         
Loyal customer    10.386336  328.017787  1.0  99999.0
New customer      13.294670  597.560299  1.0  99999.0
Regular customer  12.495717  539.720919  1.0  99999.0

>It seems that the loyal customers tend to buy more products with the lowest prices and new customer products with the highest prices.<br>
But looking at the max values, there seems to be something odd. Let's examine the value counts

Check consistency of price column

In [10]:
# show value counts sorted by prices
df.prices.value_counts().sort_index()

1.0        115143
1.1        242782
1.2        256258
1.3        425691
1.4        392209
            ...  
24.8          410
24.9         5781
25.0        13772
14900.0      4429
99999.0       698
Name: prices, Length: 242, dtype: int64

>Valid price values seem to be in the range from 1.0 to 25.0 and are incremented in steps of 0.1<br>
99999 likely represents missing data, which can potentially skew our results.<br>
14900 is not clear. Maybe it's a typo and should be 14.9.<br>

Perform data cleaning on price column

In [11]:
df.loc[df["prices"] > 25] = np.nan

Check again

In [12]:
# prices summary for different customer groups >> for valid prices only
df.groupby("loyalty_flag").agg({"prices": ["mean", "std", "min", "max"]})

prices                     
                      mean       std  min   max
loyalty_flag                                   
Loyal customer    7.773575  4.206151  1.0  25.0
New customer      7.801206  4.266834  1.0  25.0
Regular customer  7.798262  4.254886  1.0  25.0

>If we drop the invalid price records, we see that there is no real difference in the mean prices the different customer groups spend on products.

##### Spending flag [#](#table-of-content)

Determine High spenders (average price > 10) and Low spenders (agerave price <=10)

In [13]:
# create column for average prices users spend on products
df["avg_usr_prices"] = df.groupby("user_id")["prices"].transform("mean")
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg_usr_prices
0,2539329.0,1.0,1.0,2.0,8.0,0.0,196.0,1.0,0.0,Soda,77.0,7.0,9.0,Mid-range product,Regulary busy,Average orders,10.0,New customer,6.367797
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Least busy days,Fewest orders,10.0,New customer,6.367797
2,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Least busy days,Average orders,10.0,New customer,6.367797
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Least busy days,Fewest orders,10.0,New customer,6.367797
4,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Least busy days,Average orders,10.0,New customer,6.367797


In [14]:
# create flag for different user spending type
da.create_flag(df, "avg_usr_prices", "spending_flag", [10], ["Low spender", "High spender"])
df["spending_flag"].value_counts()

Low spender     32280394
High spender      119338
Name: spending_flag, dtype: int64

##### Order frequency flag [#](#table-of-content)

In [15]:
# create column with median days since last order by users
df["median_days_since_prior_order"] = df.groupby("user_id")["days_since_prior_order"].transform("median")
# create order frequency flag column
da.create_flag(df, "median_days_since_prior_order", "order_freq_flag", [10, 20], ["Frequent customer", "Regular customer", "Non-frequent customer"])

# show value counts
df["order_freq_flag"].value_counts()

Frequent customer        22793177
Regular customer          6920582
Non-frequent customer     2685973
Name: order_freq_flag, dtype: int64

## Export [#](#table-of-content)

In [16]:
df.to_pickle(orders_products_merged_updated_v2_data_file)

In [17]:
df.shape

(32404859, 22)